In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
Ideas:
- ampliar rango dinamico 
- Podriamos hacer homogenizar el histograma
- Filtro de suavizado 
- Filtro de sal y pimienta(sobre todo sal y pimienta)

In [ ]:
como subir documentos
git pull
modificar
git add . ( o nombre del archivo)
git commit -m "descripcion de los cambios realizados"

In [21]:
imagen = cv2.imread('cat_dog_100/train/dog/dog.10002.jpg', 0)
print(np.min(imagen), np.max(imagen));
imagenAmpliada = ampliarRangoDin(imagen)
cv2.imshow('imagen original', imagen)
cv2.imshow('imagen ampliada', imagenAmpliada)
cv2.waitKey(0)
cv2.destroyAllWindows()

2 253


In [19]:
def ampliarRangoDin(imagen):
    imagen = np.float32(imagen);
    imagenPlana = imagen.reshape(1, -1)
    pixelesOrd = np.sort(imagenPlana)
    al = np.unique(pixelesOrd)[5]
    ah = np.unique(pixelesOrd)[-6]
    imagenAmpliada = (imagen - al) * (255 /(ah - al))
    np.clip(imagenAmpliada, 0, 255)
    imagenAmpliada = np.uint8(np.clip(imagenAmpliada, 0, 255))
    return imagenAmpliada

In [58]:
from scipy.ndimage import median_filter
def filtroMediana (imagen, n = None):
    imagen = np.float32(imagen)
    if (n is None):
        n = 3
    ndiv = n //2
    imagen_filtrada = np.zeros_like(imagen)
    imagen_ampliada = cv2.copyMakeBorder(imagen, ndiv, ndiv, ndiv, ndiv, cv2.BORDER_REPLICATE)
    ventana = n * 2 + 1
    imagen_filtrada = median_filter(imagen, size=ventana)

    imagen_filtrada = np.uint8(imagen_filtrada)
    return imagen_filtrada
    

In [97]:
from skimage.util import random_noise

imagen = cv2.imread('cat_dog_100/train/dog/dog.10095.jpg', 0)
imagen_sp = np.uint8(255 * random_noise(imagen, mode= 's&p', amount = 0.1))
imagenFiltradaMedia = (filtroMediana(imagen_sp))
cv2.imshow('imagen original', imagen_sp)
cv2.imshow('imagen filtrada', imagenFiltradaMedia)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#Bordes 

In [14]:
def filtro(imagen, mascara):
    imagen = np.float32(imagen)
    imagen_f = cv2.filter2D(imagen, -1, mascara)
    return imagen_f

In [15]:
def gradiente(imagen):
    m_g_x = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]] )
    m_g_y = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    Ix = filtro(imagen, m_g_x)
    Iy = filtro(imagen, m_g_y)
    E = np.sqrt(Ix**2 + Iy**2)
    Phi = np.arctan2(Ix, Iy)
    return E, Phi

In [6]:
imagen = cv2.imread('cat_dog_100/train/dog/dog.10095.jpg', 0)
E, phi = gradiente(imagen)

cv2.imshow('Magnitudes',E/E.max())
cv2.waitKey(0)
cv2.destroyAllWindows()

In [67]:
import numpy as np
import cv2 as cv

filename = 'cat_dog_100/train/dog/dog.10009.jpg'
img = cv.imread(filename)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

gray = np.float32(gray)
gray = filtroMediana(gray)
dst = cv.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]= 255

cv.imshow('dst',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
Probar a umbralizar, como en la imagen del platano, sino hay mucha mierda

In [7]:
def imagen_media_chat(imagen, etiquetas, centros):
    print(centros)
    etiquetas = etiquetas.reshape(imagen.shape)
    imagen_segmentada = np.zeros_like(imagen, dtype=np.float32)    
    for i, centro in enumerate(centros):
        imagen_segmentada[etiquetas == i] = centro
    
    return imagen_segmentada.astype(np.uint8)  # Convertir a uint8 para visualización


In [9]:
from sklearn.cluster import KMeans
imagen = cv2.imread('cat_dog_100/train/dog/dog.10009.jpg', 0)
kmeans = KMeans(n_clusters = 5, n_init=10)
kmeans.fit(imagen.reshape(-1, 1))
centros = kmeans.cluster_centers_
etiquetas = kmeans.labels_
imagenMedia = imagen_media_chat(imagen, etiquetas, centros)
cv2.imshow('Imagen media', imagenMedia)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[111.47733756]
 [169.51025449]
 [220.94699711]
 [137.97871623]
 [ 69.55864902]]
